# DATA MINING PROJECT: Analysis of a Supermarket’s Customers
## 4) Pattern Mining
### *Antonio Strippoli, Valerio Mariani*

In [1]:
from gsp import apriori
import pandas as pd
import pickle
import time
import logging
import os

# Set logging
if os.path.exists('log.txt'):
    os.remove('log.txt')
logging.basicConfig(level=logging.INFO, filename="log.txt", filemode="a+", format="%(message)s")
logging.getLogger().addHandler(logging.StreamHandler())

### Functions

In [2]:
def read_dataset():
    """Read the dataset using Pandas, keeping only bought items."""
    df = pd.read_csv("../DM_25_TASK1/customer_supermarket_2.csv", index_col=0, parse_dates=["PurchaseDate"])
    return df[df['Qta'] > 0]

In [3]:
def remove_baskets(df, threshold):
    """Keep only customers with more than `threshold` baskets."""
    customers = df.groupby('CustomerID').agg({'BasketID': 'nunique'})
    customers = customers[customers >= threshold].dropna().index.values
    return df[df['CustomerID'].isin(customers)]

In [4]:
# Convert data in sequential form
def get_sequential_form(df):
    """Convert a dataset into its sequential form."""
    seq_data = []
    for customer in df.groupby('CustomerID'):
        customer = customer[1]
        tmp = []
        for basket in customer.groupby('BasketID'):
            basket = basket[1]
            purchases = list( basket['ProdID'].unique() )
            tmp.append(purchases)
        seq_data.append(tmp)
    return seq_data

In [5]:
def save_to_pickle(result_set, min_baskets, min_sup):
    """Save gsp results"""
    with open(f'gsp_res/{min_baskets}mb_{int(min_sup*100)}ms.pickle', 'wb') as handle:
        pickle.dump(result_set, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Apply GSP on sequential data

In [6]:
# Main cycle: apply GSP multiple times
params = {
    'min_sup': [0.4, 0.35, 0.3, 0.25, 0.2, 0.15],
    'min_baskets': [20, 10, 5, 3, 2],
}
for min_sup in params['min_sup']:
    for min_baskets in params['min_baskets']:
        logging.info(f"MIN_BASKETS: {min_baskets}, MIN_SUP: {min_sup}")

        # Read the dataset
        df = read_dataset()
        # Remove some baskets
        df = remove_baskets(df, min_baskets)
        # Convert into seq form
        seq_data = get_sequential_form(df)
        
        # Apply GSP
        t0 = time.time()
        result_set = apriori(seq_data, min_sup, verbose=False)
        t1 = time.time()

        # Compute n. of sequences with len > 2
        cnt_len_2 = 0
        cnt_duplicates = 0
        for r in result_set:
            r = r[0]
            tmp = []
            for l in r:
                tmp.extend(l)
            if len(tmp) >= 2:
                cnt_len_2 += 1
                if len(set(tmp)) < len(tmp):
                    cnt_duplicates += 1

        logging.info(
            f"TOTAL TIME:\t{round(t1-t0, 2)} s\n"\
            f"LEN RESULT SET:\t{len(result_set)}\n"\
            f"LEN SEQ > 2:\t{cnt_len_2}\nN. DUPLICATES:\t{cnt_duplicates}\n"
        )

        # Save
        save_to_pickle(result_set, min_baskets, min_sup)

MIN_SUP: 0.4, MIN_BASKETS: 20
TOTAL TIME:	12.95 s
LEN RESULT SET:	24
LEN SEQ > 2:	8
N. DUPLICATES:	7

MIN_SUP: 0.4, MIN_BASKETS: 10
TOTAL TIME:	29.86 s
LEN RESULT SET:	4
LEN SEQ > 2:	0
N. DUPLICATES:	0

MIN_SUP: 0.4, MIN_BASKETS: 5
TOTAL TIME:	52.25 s
LEN RESULT SET:	0
LEN SEQ > 2:	0
N. DUPLICATES:	0

MIN_SUP: 0.4, MIN_BASKETS: 3
TOTAL TIME:	70.58 s
LEN RESULT SET:	0
LEN SEQ > 2:	0
N. DUPLICATES:	0

MIN_SUP: 0.4, MIN_BASKETS: 2
TOTAL TIME:	81.14 s
LEN RESULT SET:	0
LEN SEQ > 2:	0
N. DUPLICATES:	0

MIN_SUP: 0.35, MIN_BASKETS: 20
TOTAL TIME:	18.2 s
LEN RESULT SET:	63
LEN SEQ > 2:	29
N. DUPLICATES:	17

MIN_SUP: 0.35, MIN_BASKETS: 10
TOTAL TIME:	30.35 s
LEN RESULT SET:	10
LEN SEQ > 2:	0
N. DUPLICATES:	0

MIN_SUP: 0.35, MIN_BASKETS: 5
TOTAL TIME:	54.5 s
LEN RESULT SET:	1
LEN SEQ > 2:	0
N. DUPLICATES:	0

MIN_SUP: 0.35, MIN_BASKETS: 3
TOTAL TIME:	73.72 s
LEN RESULT SET:	0
LEN SEQ > 2:	0
N. DUPLICATES:	0

MIN_SUP: 0.35, MIN_BASKETS: 2
TOTAL TIME:	79.95 s
LEN RESULT SET:	0
LEN SEQ > 2:	0
N. DUP

KeyboardInterrupt: 

### Considerazioni

Prendendo quelli che hanno fatto almeno 20 baskets, otteniamo una mole maggiore di risultati. Abbassando min_baskets, il supporto comincia a diventare sempre più basso in generale, il che lascia comunque presagire che nei clienti più occasionali non ci siano pattern evidenti.

Il parametro min_baskets è importante perché altrimenti trovare dei pattern un po' più interessanti (che spazino tra basket diversi) è molto difficile (richiedono min_support bassi che alzano il costo computazionale). Alla fine abbiamo scelto 10 come giusto compromesso tra un numero di clienti abbastanza alto (il 10% di quelli di partenza) e sequenze variegate/lunghe.

### Collect statistics

In [27]:
# Read gsp results
with open('gsp_res/10mb_25ms.pickle', 'rb') as handle:
    result_set = pickle.load(handle)

result_set.sort(key=lambda x: x[1], reverse=True)

# Convert ProductIDs to readable descriptions
result_set_desc = []
df = read_dataset()
for r_i, result in enumerate(result_set):
    tmp = []
    for b_i, basket in enumerate(result[0]):
        tmp2 = []
        for p_i, p in enumerate(basket):
            tmp2.append(df[df['ProdID'] == p]['ProdDescr'].iloc[0])
        tmp.append(tmp2)
    result_set_desc.append((tmp, result[1]))
result_set_desc

[([['REGENCY CAKESTAND 3 TIER']], 0.42),
 ([['JUMBO BAG RED RETROSPOT']], 0.42),
 ([['PACK OF 72 RETROSPOT CAKE CASES']], 0.4),
 ([['PARTY BUNTING']], 0.4),
 ([['LUNCH BAG RED SPOTTY']], 0.39),
 ([['WHITE HANGING HEART T-LIGHT HOLDER']], 0.39),
 ([['SET OF 3 CAKE TINS PANTRY DESIGN']], 0.38),
 ([['JUMBO BAG VINTAGE DOILY']], 0.37),
 ([['LUNCH BAG VINTAGE DOILY']], 0.35),
 ([['SPOTTY BUNTING']], 0.35),
 ([['ASSORTED COLOUR BIRD ORNAMENT']], 0.34),
 ([['REGENCY CAKESTAND 3 TIER'], ['REGENCY CAKESTAND 3 TIER']], 0.33),
 ([['JUMBO BAG RED RETROSPOT'], ['JUMBO BAG RED RETROSPOT']], 0.33),
 ([['WHITE HANGING HEART T-LIGHT HOLDER'],
   ['WHITE HANGING HEART T-LIGHT HOLDER']],
  0.33),
 ([['LUNCH BAG  BLACK SKULL.']], 0.32),
 ([['SET OF 3 REGENCY CAKE TINS']], 0.32),
 ([['LUNCH BAG SPACEBOY DESIGN']], 0.31),
 ([['LUNCH BAG PINK POLKADOT']], 0.31),
 ([['SET OF 4 PANTRY JELLY MOULDS']], 0.31),
 ([['LUNCH BAG ALPHABET DESIGN']], 0.31),
 ([['LUNCH BAG CARS BLUE']], 0.3),
 ([['SET/20 RED RETROSPOT 

### Considerazioni su RESULT_SET
- Conta sequenze che hanno doppioni e quelle che non ce l'hanno (in percentuale?)
- ogni evento è formato da un singolo elemento (non ci sono carrelli che hanno in comune più di un elemento, ma solo sequenze di carrelli con oggetti comuni)
- Contare quante sono sequenze lunghe 1, 2, 3...

### Considerazioni su transazioni e clienti a cui si riferiscono le sequenze
- quantità media di oggetti presa per ogni oggetto di ogni sequenza
- tempo passato tra una transazione e l'altra

### Considerazioni
due casi:
- oggetti uguali: l'oggetto vende bene? Vengono ricomprati per essere venduti ancora
- oggetti diversi: oggetto1 magari non ha venduto benissimo e se n'è comprato uno simile per provare a vendere quello, oppure ha comprato semplicemente un'altra variante.

In [28]:
len(remove_baskets(df, 1)['CustomerID'].unique())

4336